In [1]:
!pip3 install pyspark

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from pyspark.sql import SparkSession
import pyspark

spark = SparkSession.builder.appName("example").getOrCreate()


25/05/15 19:36:09 WARN Utils: Your hostname, LAPTOP-UNBDFQLR resolves to a loopback address: 127.0.1.1; using 172.23.167.235 instead (on interface eth0)
25/05/15 19:36:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/15 19:36:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


####Data Cleaning

In [3]:
stocks = spark.read.csv("/home/rheas/data/StockData/*.csv", header=True, inferSchema=True)
stocks.show(5)

+------+----------+----------+-------+--------+--------+--------+
|Ticker|      Date|Close/Last| Volume|    Open|    High|     Low|
+------+----------+----------+-------+--------+--------+--------+
| BRK-B|05/31/2023|  $321.08 |6175417|$321.12 |$322.41 |$319.39 |
| BRK-B|05/30/2023|  $322.19 |3232461|$321.86 |$322.47 |$319.00 |
| BRK-B|05/26/2023|  $320.60 |3229873|$320.44 |$322.63 |$319.67 |
| BRK-B|05/25/2023|  $319.02 |4251935|$320.56 |$320.56 |$317.71 |
| BRK-B|05/24/2023|  $320.20 |3075393|$322.71 |$323.00 |$319.56 |
+------+----------+----------+-------+--------+--------+--------+
only showing top 5 rows



In [4]:
stocks.printSchema()

root
 |-- Ticker: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Close/Last: string (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)



In [5]:
stocks.filter(stocks.Ticker=='MSFT').show(10)

+------+----------+----------+--------+--------+--------+--------+
|Ticker|      Date|Close/Last|  Volume|    Open|    High|     Low|
+------+----------+----------+--------+--------+--------+--------+
|  MSFT|05/31/2023|  $328.39 |45950550|$332.29 |$335.94 |$327.33 |
|  MSFT|05/30/2023|  $331.21 |29503070|$335.23 |$335.74 |$330.52 |
|  MSFT|05/26/2023|  $332.89 |36630630|$324.02 |$333.40 |$323.88 |
|  MSFT|05/25/2023|  $325.92 |43301740|$323.24 |$326.90 |$320.00 |
|  MSFT|05/24/2023|  $313.85 |23384890|$314.73 |$316.50 |$312.61 |
|  MSFT|05/23/2023|  $315.26 |30797170|$320.03 |$322.72 |$315.25 |
|  MSFT|05/22/2023|  $321.18 |24115660|$318.60 |$322.59 |$318.01 |
|  MSFT|05/19/2023|  $318.34 |27546700|$316.74 |$318.75 |$316.37 |
|  MSFT|05/18/2023|  $318.52 |27275990|$314.53 |$319.04 |$313.72 |
|  MSFT|05/17/2023|  $314.00 |24315010|$312.29 |$314.43 |$310.74 |
+------+----------+----------+--------+--------+--------+--------+
only showing top 10 rows



In [6]:
stocks.filter(((stocks.Ticker=='MSFT') | (stocks.Ticker=='V')) & (stocks.Date=='05/31/2023')).show(10)

+------+----------+----------+--------+--------+--------+--------+
|Ticker|      Date|Close/Last|  Volume|    Open|    High|     Low|
+------+----------+----------+--------+--------+--------+--------+
|  MSFT|05/31/2023|  $328.39 |45950550|$332.29 |$335.94 |$327.33 |
|     V|05/31/2023|  $221.03 |20460620|$219.96 |$221.53 |$216.14 |
+------+----------+----------+--------+--------+--------+--------+



In [7]:
stocks.filter((stocks.Ticker.isin(["MSFT","QQQ", "SPY","V","TSLA"])) & (stocks.Date=='05/31/2023')).show(10)

+------+----------+----------+---------+--------+--------+--------+
|Ticker|      Date|Close/Last|   Volume|    Open|    High|     Low|
+------+----------+----------+---------+--------+--------+--------+
|  MSFT|05/31/2023|  $328.39 | 45950550|$332.29 |$335.94 |$327.33 |
|  TSLA|05/31/2023|  $203.93 |150711700|$199.78 |$203.95 |$195.12 |
|     V|05/31/2023|  $221.03 | 20460620|$219.96 |$221.53 |$216.14 |
|   SPY|05/31/2023|    417.85|110811800|  418.28|  419.22|  416.22|
|   QQQ|05/31/2023|    347.99| 65105380|  348.37|   350.6|  346.51|
+------+----------+----------+---------+--------+--------+--------+



In [8]:
from pyspark.sql.functions import udf
from pyspark.sql.types import DateType


In [9]:
from datetime import datetime

In [10]:
date_parser = udf(lambda date: datetime.strptime(date,"%m/%d/%Y"), DateType())

In [11]:
stocks.withColumn("ParsedDate", date_parser(stocks.Date)).show(5)

+------+----------+----------+-------+--------+--------+--------+----------+
|Ticker|      Date|Close/Last| Volume|    Open|    High|     Low|ParsedDate|
+------+----------+----------+-------+--------+--------+--------+----------+
| BRK-B|05/31/2023|  $321.08 |6175417|$321.12 |$322.41 |$319.39 |2023-05-31|
| BRK-B|05/30/2023|  $322.19 |3232461|$321.86 |$322.47 |$319.00 |2023-05-30|
| BRK-B|05/26/2023|  $320.60 |3229873|$320.44 |$322.63 |$319.67 |2023-05-26|
| BRK-B|05/25/2023|  $319.02 |4251935|$320.56 |$320.56 |$317.71 |2023-05-25|
| BRK-B|05/24/2023|  $320.20 |3075393|$322.71 |$323.00 |$319.56 |2023-05-24|
+------+----------+----------+-------+--------+--------+--------+----------+
only showing top 5 rows



In [12]:
def num_parser(value):
    if isinstance(value, str):
        return float(value.strip("$"))
    elif isinstance(value,int) or isinstance(value, float):
        return value
    else:
        return None
from pyspark.sql.types import FloatType
parser_number = udf(num_parser, FloatType())

In [13]:
stocks=(stocks.withColumn("Open", parser_number(stocks.Open))
               .withColumn("High", parser_number(stocks.High))
                .withColumn("Low", parser_number(stocks.Low))
                 .withColumn("Close", parser_number(stocks["Close/Last"]))
                  .withColumn("Date", date_parser(stocks.Date)))

In [14]:
from pyspark.sql.types import IntegerType

In [15]:
stocks.show(5)

+------+----------+----------+-------+------+------+------+------+
|Ticker|      Date|Close/Last| Volume|  Open|  High|   Low| Close|
+------+----------+----------+-------+------+------+------+------+
| BRK-B|2023-05-31|  $321.08 |6175417|321.12|322.41|319.39|321.08|
| BRK-B|2023-05-30|  $322.19 |3232461|321.86|322.47| 319.0|322.19|
| BRK-B|2023-05-26|  $320.60 |3229873|320.44|322.63|319.67| 320.6|
| BRK-B|2023-05-25|  $319.02 |4251935|320.56|320.56|317.71|319.02|
| BRK-B|2023-05-24|  $320.20 |3075393|322.71| 323.0|319.56| 320.2|
+------+----------+----------+-------+------+------+------+------+
only showing top 5 rows



In [16]:
parse_int = udf(lambda value: int(value), IntegerType())
cleaned_stocks=stocks.withColumn("Volume", parse_int(stocks.Volume))

In [17]:
cleaned_stocks.describe().show()

25/05/15 19:36:19 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
[Stage 13:>                                                       (0 + 12) / 12]

+-------+------+------------------+--------------------+------------------+------------------+------------------+------------------+
|summary|Ticker|        Close/Last|              Volume|              Open|              High|               Low|             Close|
+-------+------+------------------+--------------------+------------------+------------------+------------------+------------------+
|  count| 15108|             15108|               15108|             15108|             15108|             15108|             15108|
|   mean|  NULL|310.52084193804643|5.1868408793685466E7|180.09656566181036| 182.1253348687101| 177.9982781513109| 180.1256089860054|
| stddev|  NULL| 81.44943282256573| 5.496484129953464E7|101.16125813324399|101.96625521621753|100.26590135955216|101.14891782168563|
|    min|  AAPL|          $100.00 |              961133|             12.07|             12.45|              11.8|             11.93|
|    max|     V|            477.71|           914080943|            4

### Basic Analysis

In [18]:
cleaned_stocks.groupBy("Ticker").max("Open").show(15)

+------+---------+
|Ticker|max(Open)|
+------+---------+
| BRK-B|   361.39|
|  MSFT|   344.62|
|  META|   381.68|
|  TSLA|   411.47|
|  AAPL|   182.63|
|  AMZN|    187.2|
| GOOGL|   151.25|
|  NVDA|   405.95|
|     V|   250.05|
|   TSM|   141.61|
|   SPY|   479.22|
|   QQQ|   405.57|
+------+---------+



In [19]:
cleaned_stocks.groupBy("Ticker").max("Open").withColumnRenamed("max(Open)","MaxStockPrice").show(15)

+------+-------------+
|Ticker|MaxStockPrice|
+------+-------------+
| BRK-B|       361.39|
|  MSFT|       344.62|
|  META|       381.68|
|  TSLA|       411.47|
|  AAPL|       182.63|
|  AMZN|        187.2|
| GOOGL|       151.25|
|  NVDA|       405.95|
|     V|       250.05|
|   TSM|       141.61|
|   SPY|       479.22|
|   QQQ|       405.57|
+------+-------------+



In [20]:
import pyspark.sql.functions as func


In [21]:
cleaned_stocks.groupBy("Ticker").agg(
    func.max("Open").alias("MaxStockPrice"),
    func.max("Volume").alias("TotalVolume")
).show(15)

+------+-------------+-----------+
|Ticker|MaxStockPrice|TotalVolume|
+------+-------------+-----------+
| BRK-B|       361.39|   22303500|
|  MSFT|       344.62|  110945000|
|  META|       381.68|  232316600|
|  TSLA|       411.47|  914080943|
|  AAPL|       182.63|  426884800|
|  AMZN|        187.2|  311345600|
| GOOGL|       151.25|  132345540|
|  NVDA|       405.95|  250920160|
|     V|       250.05|   38379570|
|   TSM|       141.61|   60793170|
|   SPY|       479.22|  392220700|
|   QQQ|       405.57|  199448100|
+------+-------------+-----------+



In [22]:
cleaned_stocks =(cleaned_stocks.withColumn("Year", func.year(cleaned_stocks.Date))
                               .withColumn("Month", func.month(cleaned_stocks.Date))
                               .withColumn("Day", func.day(cleaned_stocks.Date))
                                .withColumn("Week", func.weekofyear(cleaned_stocks.Date)))

In [23]:
cleaned_stocks.show(10)

+------+----------+----------+-------+------+------+------+------+----+-----+---+----+
|Ticker|      Date|Close/Last| Volume|  Open|  High|   Low| Close|Year|Month|Day|Week|
+------+----------+----------+-------+------+------+------+------+----+-----+---+----+
| BRK-B|2023-05-31|  $321.08 |6175417|321.12|322.41|319.39|321.08|2023|    5| 31|  22|
| BRK-B|2023-05-30|  $322.19 |3232461|321.86|322.47| 319.0|322.19|2023|    5| 30|  22|
| BRK-B|2023-05-26|  $320.60 |3229873|320.44|322.63|319.67| 320.6|2023|    5| 26|  21|
| BRK-B|2023-05-25|  $319.02 |4251935|320.56|320.56|317.71|319.02|2023|    5| 25|  21|
| BRK-B|2023-05-24|  $320.20 |3075393|322.71| 323.0|319.56| 320.2|2023|    5| 24|  21|
| BRK-B|2023-05-23|  $323.11 |4031342|328.19|329.27|322.97|323.11|2023|    5| 23|  21|
| BRK-B|2023-05-22|  $329.13 |2763422|330.75|331.49|328.35|329.13|2023|    5| 22|  21|
| BRK-B|2023-05-19|  $330.39 |4323538| 331.0|333.94|329.12|330.39|2023|    5| 19|  20|
| BRK-B|2023-05-18|  $329.76 |2808329|326.8

In [24]:
yearly =cleaned_stocks.groupBy(['Ticker','Year']).agg( func.max("Open").alias("YearlyHigh"),
                                                       func.min("Open").alias("YearlyLow"))

In [25]:
yearly.show()

+------+----+----------+---------+
|Ticker|Year|YearlyHigh|YearlyLow|
+------+----+----------+---------+
| BRK-B|2023|     331.0|   294.68|
| BRK-B|2018|     224.0|   185.43|
| BRK-B|2021|    300.88|   228.21|
| BRK-B|2020|    233.92|    165.3|
| BRK-B|2019|    227.27|   194.78|
| BRK-B|2022|    361.39|   260.58|
|  MSFT|2019|    159.45|    99.55|
|  MSFT|2021|    344.62|   212.17|
|  MSFT|2020|    229.27|   137.01|
|  MSFT|2018|    115.42|    95.14|
|  MSFT|2023|    335.23|    223.0|
|  MSFT|2022|    335.35|   217.55|
|  META|2020|    300.16|   139.75|
|  META|2021|    381.68|    247.9|
|  META|2018|    215.72|    123.1|
|  META|2022|    339.95|    90.08|
|  META|2019|    208.67|   128.99|
|  META|2023|    265.25|   122.82|
|  TSLA|2019|      29.0|    12.07|
|  TSLA|2021|    411.47|   184.18|
+------+----+----------+---------+
only showing top 20 rows



In [26]:
weekly =cleaned_stocks.groupBy(['Ticker','Year','Week']).agg( func.max("Open").alias("WeeklyHigh"),
                                                       func.min("Open").alias("WeeklyLow"))

In [27]:
weekly.show()

+------+----+----+----------+---------+
|Ticker|Year|Week|WeeklyHigh|WeeklyLow|
+------+----+----+----------+---------+
| BRK-B|2022|  14|     352.0|   341.17|
| BRK-B|2022|  10|    326.59|   322.49|
| BRK-B|2021|  14|    264.22|   260.02|
| BRK-B|2018|  48|    217.23|    209.3|
| BRK-B|2020|  19|    180.05|    173.4|
| BRK-B|2021|  32|    291.81|   287.01|
| BRK-B|2021|  10|    264.22|    255.6|
| BRK-B|2020|  10|    217.39|   203.48|
| BRK-B|2020|  28|     183.1|   178.26|
| BRK-B|2019|  30|    208.37|   205.34|
| BRK-B|2019|  23|    204.95|   197.62|
| BRK-B|2018|  40|     219.7|   214.43|
| BRK-B|2022|  40|     278.2|   269.52|
| BRK-B|2022|  38|     278.9|   266.01|
| BRK-B|2020|  52|     224.6|   222.02|
| BRK-B|2019|   9|    203.15|    201.7|
| BRK-B|2021|   6|    241.43|    236.0|
| BRK-B|2022|  11|    344.12|   328.65|
| BRK-B|2018|  28|     190.9|   188.11|
| BRK-B|2022|  28|    279.38|    273.0|
+------+----+----+----------+---------+
only showing top 20 rows



In [28]:
Monthly =cleaned_stocks.groupBy(['Ticker','Year','Month']).agg( func.max("Open").alias("MonthlyHigh"),
                                                       func.min("Open").alias("MonthlyLow"))

In [29]:
Monthly.show()

+------+----+-----+-----------+----------+
|Ticker|Year|Month|MonthlyHigh|MonthlyLow|
+------+----+-----+-----------+----------+
| BRK-B|2022|   10|     297.98|    260.58|
| BRK-B|2018|    9|     222.13|    209.21|
| BRK-B|2021|   10|     290.85|    273.02|
| BRK-B|2020|   10|     216.74|    200.03|
| BRK-B|2019|    9|     212.24|    201.19|
| BRK-B|2021|    6|     292.91|     275.0|
| BRK-B|2022|   11|     317.52|    286.02|
| BRK-B|2022|    7|     297.42|     272.5|
| BRK-B|2022|    6|      316.0|    267.38|
| BRK-B|2020|    8|     218.62|    197.28|
| BRK-B|2021|   12|     300.88|    276.68|
| BRK-B|2019|    1|     204.68|    194.78|
| BRK-B|2019|    6|      213.0|    197.62|
| BRK-B|2020|    7|     195.06|    178.26|
| BRK-B|2021|    8|     291.81|    279.05|
| BRK-B|2018|    6|      196.3|    185.43|
| BRK-B|2022|    5|     325.85|    303.93|
| BRK-B|2020|   12|     231.26|    222.02|
| BRK-B|2020|    4|      194.0|     175.0|
| BRK-B|2018|    8|     211.08|     197.2|
+------+---

In [30]:
weekly.withColumn("Spread", weekly["WeeklyHigh"]-weekly["WeeklyLow"]).show()

+------+----+----+----------+---------+---------+
|Ticker|Year|Week|WeeklyHigh|WeeklyLow|   Spread|
+------+----+----+----------+---------+---------+
| BRK-B|2022|  14|     352.0|   341.17|10.829987|
| BRK-B|2022|  10|    326.59|   322.49| 4.100006|
| BRK-B|2021|  14|    264.22|   260.02| 4.200012|
| BRK-B|2018|  48|    217.23|    209.3|7.9299927|
| BRK-B|2020|  19|    180.05|    173.4| 6.650009|
| BRK-B|2021|  32|    291.81|   287.01| 4.799988|
| BRK-B|2021|  10|    264.22|    255.6| 8.619995|
| BRK-B|2020|  10|    217.39|   203.48|13.910004|
| BRK-B|2020|  28|     183.1|   178.26|4.8400116|
| BRK-B|2019|  30|    208.37|   205.34|3.0299988|
| BRK-B|2019|  23|    204.95|   197.62| 7.330002|
| BRK-B|2018|  40|     219.7|   214.43|5.2700043|
| BRK-B|2022|  40|     278.2|   269.52| 8.680023|
| BRK-B|2022|  38|     278.9|   266.01|12.889984|
| BRK-B|2020|  52|     224.6|   222.02|2.5800018|
| BRK-B|2019|   9|    203.15|    201.7| 1.449997|
| BRK-B|2021|   6|    241.43|    236.0|5.4299927|


In [31]:
historic_stocks = cleaned_stocks.join(yearly, 
                  (cleaned_stocks.Ticker ==yearly.Ticker) & (cleaned_stocks.Year ==yearly.Year),
                 'inner'
                   ).drop(yearly.Year, yearly.Ticker)

In [32]:
historic_stocks.show()

+----------+----------+-------+------+------+------+------+-----+---+----+------+----+----------+---------+
|      Date|Close/Last| Volume|  Open|  High|   Low| Close|Month|Day|Week|Ticker|Year|YearlyHigh|YearlyLow|
+----------+----------+-------+------+------+------+------+-----+---+----+------+----+----------+---------+
|2023-05-31|  $321.08 |6175417|321.12|322.41|319.39|321.08|    5| 31|  22| BRK-B|2023|     331.0|   294.68|
|2023-05-30|  $322.19 |3232461|321.86|322.47| 319.0|322.19|    5| 30|  22| BRK-B|2023|     331.0|   294.68|
|2023-05-26|  $320.60 |3229873|320.44|322.63|319.67| 320.6|    5| 26|  21| BRK-B|2023|     331.0|   294.68|
|2023-05-25|  $319.02 |4251935|320.56|320.56|317.71|319.02|    5| 25|  21| BRK-B|2023|     331.0|   294.68|
|2023-05-24|  $320.20 |3075393|322.71| 323.0|319.56| 320.2|    5| 24|  21| BRK-B|2023|     331.0|   294.68|
|2023-05-23|  $323.11 |4031342|328.19|329.27|322.97|323.11|    5| 23|  21| BRK-B|2023|     331.0|   294.68|
|2023-05-22|  $329.13 |27634

In [33]:
cond =[(historic_stocks.Ticker ==weekly.Ticker) & (historic_stocks.Year ==weekly.Year)&(historic_stocks.Year ==weekly.Year)]
historic_stocks = historic_stocks.join(weekly, 
                  cond,
                 'inner'
                   ).drop(yearly.Year, yearly.Ticker).drop(weekly.Year, historic_stocks.Ticker, weekly.Week)

In [34]:
historic_stocks.show()

+----------+----------+-------+------+------+------+------+-----+---+----+----------+---------+------+----+----------+---------+
|      Date|Close/Last| Volume|  Open|  High|   Low| Close|Month|Day|Year|YearlyHigh|YearlyLow|Ticker|Week|WeeklyHigh|WeeklyLow|
+----------+----------+-------+------+------+------+------+-----+---+----+----------+---------+------+----+----------+---------+
|2023-05-31|  $321.08 |6175417|321.12|322.41|319.39|321.08|    5| 31|2023|     331.0|   294.68| BRK-B|   6|    311.12|   307.08|
|2023-05-31|  $321.08 |6175417|321.12|322.41|319.39|321.08|    5| 31|2023|     331.0|   294.68| BRK-B|   4|    312.99|   308.33|
|2023-05-31|  $321.08 |6175417|321.12|322.41|319.39|321.08|    5| 31|2023|     331.0|   294.68| BRK-B|  17|    325.99|   322.86|
|2023-05-31|  $321.08 |6175417|321.12|322.41|319.39|321.08|    5| 31|2023|     331.0|   294.68| BRK-B|  10|    316.39|   302.95|
|2023-05-31|  $321.08 |6175417|321.12|322.41|319.39|321.08|    5| 31|2023|     331.0|   294.68| B

In [36]:
historic_stocks=historic_stocks.join(Monthly, ['Ticker', 'Year','Month'])

In [39]:
historic_stocks.show()

+------+----+-----+----------+----------+-------+------+------+------+------+---+----------+---------+----+----------+---------+-----------+----------+
|Ticker|Year|Month|      Date|Close/Last| Volume|  Open|  High|   Low| Close|Day|YearlyHigh|YearlyLow|Week|WeeklyHigh|WeeklyLow|MonthlyHigh|MonthlyLow|
+------+----+-----+----------+----------+-------+------+------+------+------+---+----------+---------+----+----------+---------+-----------+----------+
| BRK-B|2023|    5|2023-05-31|  $321.08 |6175417|321.12|322.41|319.39|321.08| 31|     331.0|   294.68|   6|    311.12|   307.08|      331.0|    320.44|
| BRK-B|2023|    5|2023-05-31|  $321.08 |6175417|321.12|322.41|319.39|321.08| 31|     331.0|   294.68|   4|    312.99|   308.33|      331.0|    320.44|
| BRK-B|2023|    5|2023-05-31|  $321.08 |6175417|321.12|322.41|319.39|321.08| 31|     331.0|   294.68|  17|    325.99|   322.86|      331.0|    320.44|
| BRK-B|2023|    5|2023-05-31|  $321.08 |6175417|321.12|322.41|319.39|321.08| 31|     33

In [42]:
final_stocks = historic_stocks.select(['Ticker', 'Year',
 'Month', 'Day', 'Week', 'Volume','Open','Low','High','Close/Last', 'YearlyHigh','YearlyLow','WeeklyHigh','WeeklyLow',
 'MonthlyHigh','MonthlyLow'])

In [43]:
final_stocks.show()

+------+----+-----+---+----+-------+------+------+------+----------+----------+---------+----------+---------+-----------+----------+
|Ticker|Year|Month|Day|Week| Volume|  Open|   Low|  High|Close/Last|YearlyHigh|YearlyLow|WeeklyHigh|WeeklyLow|MonthlyHigh|MonthlyLow|
+------+----+-----+---+----+-------+------+------+------+----------+----------+---------+----------+---------+-----------+----------+
| BRK-B|2023|    5| 31|   6|6175417|321.12|319.39|322.41|  $321.08 |     331.0|   294.68|    311.12|   307.08|      331.0|    320.44|
| BRK-B|2023|    5| 31|   4|6175417|321.12|319.39|322.41|  $321.08 |     331.0|   294.68|    312.99|   308.33|      331.0|    320.44|
| BRK-B|2023|    5| 31|  17|6175417|321.12|319.39|322.41|  $321.08 |     331.0|   294.68|    325.99|   322.86|      331.0|    320.44|
| BRK-B|2023|    5| 31|  10|6175417|321.12|319.39|322.41|  $321.08 |     331.0|   294.68|    316.39|   302.95|      331.0|    320.44|
| BRK-B|2023|    5| 31|  19|6175417|321.12|319.39|322.41|  $32

In [44]:
final_stocks.createOrReplaceTempView('StockData')

In [45]:
spark.sql("Select * from stockData where Ticker ='MSFT' and Year ='2023'").show(5)

+------+----+-----+---+----+--------+------+------+------+----------+----------+---------+----------+---------+-----------+----------+
|Ticker|Year|Month|Day|Week|  Volume|  Open|   Low|  High|Close/Last|YearlyHigh|YearlyLow|WeeklyHigh|WeeklyLow|MonthlyHigh|MonthlyLow|
+------+----+-----+---+----+--------+------+------+------+----------+----------+---------+----------+---------+-----------+----------+
|  MSFT|2023|    5| 31|   6|45950550|332.29|327.33|335.94|  $328.39 |    335.23|    223.0|     273.8|   257.44|     335.23|    305.72|
|  MSFT|2023|    5| 31|   1|45950550|332.29|327.33|335.94|  $328.39 |    335.23|    223.0|    243.08|    223.0|     335.23|    305.72|
|  MSFT|2023|    5| 31|  20|45950550|332.29|327.33|335.94|  $328.39 |    335.23|    223.0|    316.74|    309.1|     335.23|    305.72|
|  MSFT|2023|    5| 31|   8|45950550|332.29|327.33|335.94|  $328.39 |    335.23|    223.0|    255.56|   249.96|     335.23|    305.72|
|  MSFT|2023|    5| 31|  17|45950550|332.29|327.33|335.

### Advanced analytics

In [48]:
snapshot= cleaned_stocks.select(['Ticker','Date', 'Open'])

In [49]:
snapshot.show()

+------+----------+------+
|Ticker|      Date|  Open|
+------+----------+------+
| BRK-B|2023-05-31|321.12|
| BRK-B|2023-05-30|321.86|
| BRK-B|2023-05-26|320.44|
| BRK-B|2023-05-25|320.56|
| BRK-B|2023-05-24|322.71|
| BRK-B|2023-05-23|328.19|
| BRK-B|2023-05-22|330.75|
| BRK-B|2023-05-19| 331.0|
| BRK-B|2023-05-18|326.87|
| BRK-B|2023-05-17|325.02|
| BRK-B|2023-05-16|322.46|
| BRK-B|2023-05-15|322.89|
| BRK-B|2023-05-12|323.82|
| BRK-B|2023-05-11| 321.0|
| BRK-B|2023-05-10|326.08|
| BRK-B|2023-05-09|324.87|
| BRK-B|2023-05-08|328.26|
| BRK-B|2023-05-05|323.36|
| BRK-B|2023-05-04|323.44|
| BRK-B|2023-05-03|327.13|
+------+----------+------+
only showing top 20 rows



In [52]:
from pyspark.sql.window import Window

In [61]:
lag1Day = Window.partitionBy("Ticker").orderBy("Date")

In [62]:
snapshot.withColumn("PreviousOpen", func.lag("Open", 1).over(lag1Day)).show()

+------+----------+-----+------------+
|Ticker|      Date| Open|PreviousOpen|
+------+----------+-----+------------+
|  AAPL|2018-05-31|46.81|        NULL|
|  AAPL|2018-06-01| 47.0|       46.81|
|  AAPL|2018-06-04|47.91|        47.0|
|  AAPL|2018-06-05|48.27|       47.91|
|  AAPL|2018-06-06|48.41|       48.27|
|  AAPL|2018-06-07|48.54|       48.41|
|  AAPL|2018-06-08|47.79|       48.54|
|  AAPL|2018-06-11|47.84|       47.79|
|  AAPL|2018-06-12|47.85|       47.84|
|  AAPL|2018-06-13|48.11|       47.85|
|  AAPL|2018-06-14|47.89|       48.11|
|  AAPL|2018-06-15|47.51|       47.89|
|  AAPL|2018-06-18|46.97|       47.51|
|  AAPL|2018-06-19|46.29|       46.97|
|  AAPL|2018-06-20|46.59|       46.29|
|  AAPL|2018-06-21|46.81|       46.59|
|  AAPL|2018-06-22|46.53|       46.81|
|  AAPL|2018-06-25|45.85|       46.53|
|  AAPL|2018-06-26|45.75|       45.85|
|  AAPL|2018-06-27|46.31|       45.75|
+------+----------+-----+------------+
only showing top 20 rows



In [64]:
movingAverage = Window.partitionBy("Ticker").orderBy("Date").rowsBetween(-50,0)

(snapshot.withColumn("MA50", func.avg("Open").over(movingAverage))
         .withColumn("MA50", func.round("MA50",2))).show()

In [66]:
maximumStock =Window.partitionBy("Ticker").orderBy(snapshot.Open.desc())

In [67]:
snapshot.withColumn("MaxOpen",func.row_number().over(maximumStock)).show()

+------+----------+------+-------+
|Ticker|      Date|  Open|MaxOpen|
+------+----------+------+-------+
|  AAPL|2022-01-04|182.63|      1|
|  AAPL|2021-12-13|181.12|      2|
|  AAPL|2021-12-28|180.16|      3|
|  AAPL|2022-01-05|179.61|      4|
|  AAPL|2021-12-30|179.47|      5|
|  AAPL|2021-12-29|179.33|      6|
|  AAPL|2021-12-16|179.28|      7|
|  AAPL|2022-03-30|178.55|      8|
|  AAPL|2021-12-31|178.09|      9|
|  AAPL|2022-03-31|177.84|     10|
|  AAPL|2022-01-03|177.83|     11|
|  AAPL|2022-04-05| 177.5|     12|
|  AAPL|2023-05-31|177.33|     13|
|  AAPL|2021-12-27|177.09|     14|
|  AAPL|2023-05-30|176.96|     15|
|  AAPL|2022-03-29|176.69|     16|
|  AAPL|2023-05-19|176.39|     17|
|  AAPL|2022-01-12|176.12|     18|
|  AAPL|2022-02-09|176.05|     19|
|  AAPL|2021-12-23|175.85|     20|
+------+----------+------+-------+
only showing top 20 rows



In [68]:
snapshot.withColumn("MaxOpen", func.row_number().over(maximumStock)).filter("MaxOpen<=5").show()

+------+----------+------+-------+
|Ticker|      Date|  Open|MaxOpen|
+------+----------+------+-------+
|  AAPL|2022-01-04|182.63|      1|
|  AAPL|2021-12-13|181.12|      2|
|  AAPL|2021-12-28|180.16|      3|
|  AAPL|2022-01-05|179.61|      4|
|  AAPL|2021-12-30|179.47|      5|
|  AMZN|2021-07-12| 187.2|      1|
|  AMZN|2021-07-09|186.13|      2|
|  AMZN|2021-07-07|185.87|      3|
|  AMZN|2021-11-19|185.63|      4|
|  AMZN|2021-07-14|185.44|      5|
| BRK-B|2022-03-29|361.39|      1|
| BRK-B|2022-03-28|360.59|      2|
| BRK-B|2022-03-31| 359.0|      3|
| BRK-B|2022-03-30|354.66|      4|
| BRK-B|2022-03-25| 353.9|      5|
| GOOGL|2022-02-02|151.25|      1|
| GOOGL|2021-11-19|149.98|      2|
| GOOGL|2021-11-08|149.83|      3|
| GOOGL|2021-11-22|149.33|      4|
| GOOGL|2021-11-09|149.23|      5|
+------+----------+------+-------+
only showing top 20 rows



In [69]:
result =snapshot.withColumn("MaxOpen", func.row_number().over(maximumStock)).filter("MaxOpen<=5")


In [73]:
moving_avg = (snapshot.withColumn("MA50", func.avg("Open").over(movingAverage))
         .withColumn("MA50", func.round("MA50", 2)))


In [74]:
result =snapshot.withColumn("MaxOpen", func.row_number().over(maximumStock)).filter("MaxOpen<=5")

In [75]:
result.show()

+------+----------+------+-------+
|Ticker|      Date|  Open|MaxOpen|
+------+----------+------+-------+
|  AAPL|2022-01-04|182.63|      1|
|  AAPL|2021-12-13|181.12|      2|
|  AAPL|2021-12-28|180.16|      3|
|  AAPL|2022-01-05|179.61|      4|
|  AAPL|2021-12-30|179.47|      5|
|  AMZN|2021-07-12| 187.2|      1|
|  AMZN|2021-07-09|186.13|      2|
|  AMZN|2021-07-07|185.87|      3|
|  AMZN|2021-11-19|185.63|      4|
|  AMZN|2021-07-14|185.44|      5|
| BRK-B|2022-03-29|361.39|      1|
| BRK-B|2022-03-28|360.59|      2|
| BRK-B|2022-03-31| 359.0|      3|
| BRK-B|2022-03-30|354.66|      4|
| BRK-B|2022-03-25| 353.9|      5|
| GOOGL|2022-02-02|151.25|      1|
| GOOGL|2021-11-19|149.98|      2|
| GOOGL|2021-11-08|149.83|      3|
| GOOGL|2021-11-22|149.33|      4|
| GOOGL|2021-11-09|149.23|      5|
+------+----------+------+-------+
only showing top 20 rows



In [77]:
(result.write.option("header", True)
            .partitionBy("Ticker","Date")
           .mode("overwrite")
           .parquet("result_parquet"))

In [78]:
(moving_avg.write.option("header", True)
            .partitionBy("Ticker","Date")
           .mode("overwrite")
           .parquet("moving_avg_parquet"))